In [1]:
import csv
import sys
import math
import os
import librosa
import numpy as np
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
sys.setrecursionlimit(10000)
import glob
import os
# reading training data
current_working_directory = os.getcwd()

base_path = current_working_directory + '/CAP6610SP24_test_set/'
prog_path = base_path + 'Other/'
# non_prog_path = base_path + 'Not_Progressive_Rock/'

def fileList(path):
    matches = glob.glob(path + '/**/*', recursive=True)
    files = [file for file in matches if os.path.isfile(file) and file.endswith(('.wav', '.mp3', '.avi', '.flac', '.ogg', '.m4a', '.wma', '.mp4'))]
    return files

prog_files = fileList(prog_path)
# non_prog_files = fileList(non_prog_path)

def get_prog_files():
    return prog_files

# def get_non_prog_files():
#     return non_prog_files

all_files = prog_files 
fixed_sample_rate = 44100


min_duration = 0
for i in range (len(all_files)):
    y, sr = librosa.load(all_files[i], sr=fixed_sample_rate)
    duration = librosa.get_duration(y=y, sr=sr)
    if i == 0:
        min_duration = duration
    else:
        min_duration = min(min_duration, duration)
min_duration = int(math.floor(min_duration))
print("Minimum Duration: ", min_duration)


Minimum Duration:  152


In [2]:
min_duration = 10

file = open('other_test_features_10.csv', 'w', newline='')
header = ['filename', 'chroma_stft', 'rmse', 'spectral_centroid', 'spectral_bandwidth', 'rolloff', 'zero_crossing_rate']

for i in range (1, 21):
    header.append('mfcc'+str(i))

# writing giles to error.txt
error_file = open('error.txt', 'w')
error_file.close()
with file:
    writer = csv.writer(file)
    writer.writerow(header)

count = 0
# time_series_length = 30


for i in range (len(prog_files)):
    filename = prog_files[i]
    name = (filename.split('/')[-1]).replace(" ", "_")

    try: 
        y, sr = librosa.load(filename, sr=fixed_sample_rate)
        time = librosa.get_duration(y=y, sr=sr)
        chunks = []
        if time > min_duration:
            org_y = y
            iter = math.floor (time /min_duration)
            print(iter)
            current_size = time*fixed_sample_rate
            chunk_size = min_duration*fixed_sample_rate
            start = 0
            end = chunk_size
            chunk_index = 1

            while iter !=0: 
                count += 1
                chunk = y[start:end]

                chroma_stft = librosa.feature.chroma_stft(y=chunk, sr=sr)
                spec_cent = librosa.feature.spectral_centroid(y=chunk, sr=sr)
                spec_bw = librosa.feature.spectral_bandwidth(y=chunk, sr=sr)
                rmse = librosa.feature.rms(y=chunk)
                rolloff = librosa.feature.spectral_rolloff(y=chunk, sr=sr)
                zcr = 10**10*np.mean(librosa.zero_crossings(org_y)/len(org_y))
                mfcc = librosa.feature.mfcc(y=chunk, sr=sr)
                to_append = f'{name+"chunk"+str(chunk_index)} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'

                # append all the 20 rows
                for e in mfcc:
                    to_append += f' {np.mean(e)}'
                
                file = open('other_test_features_10.csv', 'a', newline='')
                with file:
                    writer = csv.writer(file)
                    writer.writerow(to_append.split())

                
                # display spectrogram
                
                # spect = librosa.feature.melspectrogram(y=chunk, sr=sr, n_fft=2048, hop_length=512)
                # spect = librosa.power_to_db(spect, ref=np.max)
                # plt.figure(figsize=(12, 8))
                # plit.axis('off')
                # librosa.display.specshow(spect, y_axis='mel', fmax=8000, x_axis='time')
                # plt.show()
                
                chunk_index += 1

                start = end
                end = end + chunk_size
                iter -= 1
        else :
            count += 1
            y, sr = librosa.load(filename, sr=fixed_sample_rate, duration=min_duration)
            chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
            spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
            spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
            rmse = librosa.feature.rms(y=y)
            rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
            zcr = 10**10*np.mean(librosa.zero_crossings(y)/len(y))
            mfcc = librosa.feature.mfcc(y=y, sr=sr)
            to_append = f'{"prog"+name} {genre} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'

            # append all the 20 rows
            for e in mfcc:
                to_append += f' {np.mean(e)}'
            
            file = open('other_test_features_10.csv', 'a', newline='')
            with file:
                writer = csv.writer(file)
                writer.writerow(to_append.split())

            
            # display spectrogram
            # spect = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=2048, hop_length=512)
            # spect = librosa.power_to_db(spect, ref=np.max)
            # plt.figure(figsize=(12, 8))
            # plt.axis('off')
            # librosa.display.specshow(spect, y_axis='mel', fmax=8000, x_axis='time')
            # plt.show()
    
    except Exception as e:
        print('error handled')
        error_file = open('error.txt', 'a')
        error_file.write(filename + '\n')
        error_file.write(str(e) + '\n')
        error_file.close()
        continue
                    


43
26
25
38
30
25
43
25
37


/Users/harlow/.local/share/virtualenvs/Progect-tV540T77/lib/python3.11/site-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


28
19
31
15
24
23
37
24
26
36
20
30
30
